<img width="8%" alt="Notion.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Notion.png" style="border-radius: 15%">

# Notion - Create pages in database from dataframe
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Notion+-+Create+pages+in+database+from+dataframe:+Error+short+description">Bug report</a>

**Tags:** #notion #database #dataframe #python #create #pages

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-04-12 (Created: 2023-03-15)

**Description:** This notebook will show how to create pages in Notion database from a dataframe. It could be very usefull to kick start a new database in Notion with historical data stored in CSV, Excel or Google Sheets.

**References:**
- [Notion Drivers Naas](https://github.com/jupyter-naas/drivers/blob/main/naas_drivers/tools/notion.py)
- [Notion API Documentation](https://developers.notion.com/)

## Input

### Import libraries

In [ ]:
import naas
import pandas as pd
from naas_drivers import notion

### Setup Variables
- `notion_token`: Get your [Notion integration token](https://docs.naas.ai/drivers/notion) and share it with your database
- `df`: DataFrame as example.
- `df_key_name`: DataFrame column name to used as key
- `notion_key_name`: Notion column name to used as key
- `notion_database`: Notion Database URL

In [ ]:
# Inputs
notion_token = naas.secret.get("NOTION_TOKEN") or "YOUR_NOTION_TOKEN"
notion_key_name = "Customer"
df = pd.DataFrame({"CUSTOMER": ["Customer 1", "Customer 2", "Customer 3"]})
df_key_name = "CUSTOMER"

# Outputs
notion_database = "https://www.notion.so/asgard-group/bba2fd39a1ed457ba8e90a1104e58d13?v=39b4ecexxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

## Model

### Get dataframe

In [ ]:
print("✅ Dataframe init:", len(df))
df.head(3)

## Output

### Create pages in Notion database
If page already exists, a new page won't be created.

In [ ]:
def create_pages(
    notion_token,
    notion_database,
    notion_key_name,
    df,
    df_key_name,
):
    # Get pages
    database_id = notion_database.split("/")[-1].split("?v=")[0]
    pages = notion.connect(notion_token).database.query(database_id, query={})
    
    # Manage dataframe empty
    if len(df) == 0:
        print("🛑 Nothing to update in Notion.")
        return
    
    # Loop in data 
    for i, row in df.iterrows():
        df_key = row[df_key_name]
        print(f"➡️ {i} - Started with '{df_key}'")
        
        # Create or update page
        page_new = True
        for page in pages:
            page_temp = page.df()
            page_id = page_temp.loc[page_temp.Name == notion_key_name, "Value"].values
            if page_id == df_key:
                page_new = False
                print(f"❎ Page already exists in Notion")
                break
        try:
            if page_new:
                # Create new page in notion
                page = notion.connect(notion_token).Page.new(database_id=database_id).create()
                page.title(notion_key_name, df_key)
                
                # You
                page.update()
                print(f"✅ Page created in Notion.", '\n')
        except Exception as e:
            print(f"❌ Error creating page in Notion", e)
            print(row)
            
create_pages(
    notion_token,
    notion_database,
    notion_key_name,
    df,
    df_key_name,
)